In [ ]:
%run __init__.py

In [ ]:
#2017 Team List
team_list = [ 'ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN',
       'CLE', 'COL', 'DET', 'HOU', 'KCR', 'LAA', 'LAD', 'MIA',
       'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SDP',
       'SEA', 'SFG', 'STL', 'TBR', 'TEX', 'TOR', 'WSN']

#Create a dataframe that mirrors existing data. 
team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'record', 'div_rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_pg',
                                    'runs_ma', 'runs_allowed_ma', 'opening_day'])
    

#Use BeautifulSoup to get the data from baseball-reference.com and store them in data frame.     
for team in team_list:
    df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                   'innings', 'record', 'div_rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                   'save', 'time', 'd_or_n', 'attendance', 'streak'])
    html = urlopen('https://www.baseball-reference.com/teams/'+team+'/2017-schedule-scores.shtml')

    bs = BeautifulSoup(html)
    for game in bs.find('table', class_='sortable').findAll('tr'):
        results = []
        for element in game.find_all('td'):
            results.append(element.text)
        if len(results) == 19:
            df.loc[len(df)] = results
            
    # adjust date and convert to datetime
    df.date = df.date.str.replace(r"\(.*\)","")
    date = []
    for i in df.date:
        split = i.split(', ')
        date.append(split[1] + ', ' + '2017')
    df.date = date
                
    # rolling means for runs and runs allowed
    df['runs_pg'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
    df['runs_ma'] = df.runs.rolling(5).mean().shift()
    df['runs_allowed_ma'] = df.runs_allowed.rolling(5).mean().shift()
    df.runs_allowed_ma.fillna(df.iloc[5].runs_allowed_ma, inplace=True)
    
    # shift record, div_rank, gb, streak and fill in the first value
    df.record = df.record.shift()
    df.record.fillna('0-0', inplace=True)
    
    for col in ['div_rank', 'gb', 'streak']:
        df[col] = df[col].shift()
        df[col].fillna('0', inplace=True)
        
    # add double header dummy variable           
    df['double_header'] = [1 if ')' in x else 0 for x in df.date]
    
    # filter down to only home games
    df = df[~df['at'].str.contains('@')].reset_index(drop=True)
    
    # opening day dummy variable
    df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]
    # create one major df
    team_df = pd.concat([team_df, df]).reset_index(drop=True)


In [ ]:
team_df.to_csv('test_data.csv', index=False, encoding='utf-8')

In [3]:
data = pd.read_csv('test_data.csv')

#drop the unneeded columns
data.drop(['at', 'winning_pitcher', 'losing_pitcher','save', 'box'],axis=1,inplace=True)

#Filling NaNs in the double_header games by using different data set. 
nan_dh_df = data[(data["attendance"].isnull()) & (data["double_header"] == 1)].sort_values('date')

#most double header games only included attendance in the first game
for x in list(nan_dh_df.index):
    data['attendance'][x] = data.iloc[x + 1]['attendance']

#check with retrosheet data why non-double header game attendance is missing
data_2015 = pd.read_csv('Data/GL2015.TXT',header=None )

#both missing 2015 values are missing in other datasets, must be a collection issue. We will drop thoese rows (4)
no_nan_att_data = data[data['attendance'].notnull()]

#inings that show NaN is the full 9 inings or 9 inings with no home team batting, fill NaN with 9.0
no_nan_att_data['innings'].fillna(9.0,inplace=True)

#There's one NaN value in streak feature, it's a opening game. Therefore, we fill it with 0
no_nan_att_data[no_nan_att_data['streak'].isnull()]
no_nan_att_data['streak'].fillna(0,inplace=True)

#Win or Lose has more than w, lose or tie. Agrregate them to win, lose, tied. 
no_nan_att_data["w_or_l"].replace(to_replace = ['W-wo','W &V;','W &X;', 'W &H;'], value = ['W','W','W','W'], inplace= True)
no_nan_att_data["w_or_l"].replace(to_replace = ['L &H;','L &V;'], value =['L','L'], inplace=True)

#Record is win-lose foramt, convert it to winning%
test_rec = no_nan_att_data.record.values
temp_rec_list = []
for x in test_rec:
    temp_rec_list.append( x.split("-"))

w_percent = []
for x, y in temp_rec_list:
    if (int(x) + int (y)) != 0:
        w_percent.append(int(x)/(int(x) + int (y)))
    if (int(x) + int (y)) == 0:
        w_percent.append(0)

no_nan_att_data.record = w_percent

#games back has up + int for leading teams and postive int for trailing teams. Make negative value for trailing team.

new_gb_t = []
for x in no_nan_att_data['gb']:
    if 'up' in x:
        new_gb_t.append(x.replace('up',''))
    if 'up' not in x and 'Tied' not in x:
        new_gb_t.append('-' + x )  
    if 'Tied' in x:
        new_gb_t.append('0')

#negative sign and the space like '- 4.0' has to be changed to '-4.0' and some 0s have negative sign 

new_gb_str=[]
for x in new_gb_t:
    if x == '-0':
        new_gb_str.append(0)        
    elif '- ' in x:
        new_gb_str.append(x.replace('- ','-'))
    else:
        new_gb_str.append(x)

#set everything to float
gb_float_list = []
for x in new_gb_str:
    gb_float_list.append(float(x))

no_nan_att_data['gb']=gb_float_list

#timestamp both date and time
no_nan_att_data['date'] = pd.to_datetime(no_nan_att_data.date)
no_nan_att_data['time'] = pd.to_datetime(no_nan_att_data.time)

#convert time to minutes format
minutes_list=[]
for x in no_nan_att_data['time']:
    minutes_list.append(x.hour * 60 + x.minute)
no_nan_att_data['time']=minutes_list

#year dummies
year_dummy = pd.get_dummies(no_nan_att_data['date'].dt.year)
year_dummy.columns = list(set(no_nan_att_data['date'].dt.year))
for x in range(1990,2017):
    year_dummy[x] = [0 for x in range(len(year_dummy.index))]

#month dummies
month_dummy = pd.get_dummies(no_nan_att_data['date'].dt.month)
month_dummy.columns = ['april', 'may','june','july','aug','sep','oct']

#weekday dummies
weekday_dummy = pd.get_dummies(no_nan_att_data['date'].dt.weekday)
weekday_dummy.columns=['M', 'T', 'W', 'TH', 'F', 'SA', 'S']

#day or night game dummies
day_dummy = pd.get_dummies(no_nan_att_data['d_or_n'],prefix='time')

#convert streak to scalar value
streak_value = []
for x in list(no_nan_att_data['streak'].values):
    if type(x) != int:
        if '+' in x:
            streak_value.append(len(x))
        if '-' in x:
            streak_value.append(-len(x)) 
        if x == 0 or x == '0':
            streak_value.append(0)
    else:
        streak_value.append(0)
no_nan_att_data['streak'] = streak_value

#win or lose result dummy
win_dummy = pd.get_dummies(no_nan_att_data['w_or_l'],prefix='result')

#attendace has comma, convert it to int
no_nan_att_data['attendance'] =[x.replace(',', '') for x in list(no_nan_att_data['attendance'].values)]
no_nan_att_data['attendance']= no_nan_att_data['attendance'].astype(int)

#putting data together

final_data = pd.concat([no_nan_att_data, win_dummy,year_dummy, month_dummy,weekday_dummy, day_dummy],axis=1)

#column name year, month, weekday to create groupby and EDA
year_list = no_nan_att_data['date'].dt.year
final_data['year'] = year_list 
month_list = no_nan_att_data['date'].dt.month
final_data['month'] = month_list 
weekday_list = no_nan_att_data['date'].dt.weekday
final_data['weekday'] = weekday_list 

#drop dummied features
final_drop = final_data.drop(['w_or_l','d_or_n'],axis=1)

num_cols = ['runs', 'runs_allowed', 'innings', 'record',
       'div_rank', 'gb', 'time', 'attendance','runs_pg',
       'runs_ma', 'runs_allowed_ma','last_attendance','streak']

cate_cols = ['double_header','opening_day', 'result_L', 'result_T', 'result_W', 'march',
       'april', 'may', 'june', 'july', 'aug', 'sep', 'oct', 'M', 'T', 'W',
       'TH', 'F', 'SA', 'S', 'time_D', 'time_N','rival']

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/opt/conda

In [7]:
final_drop.to_pickle('test_data_mlb.pkl')